# How To Setup Azure Machine Learning Services

The purpose of this notebook is to concisely describe how you can
- create an Azure Machine Learning Workspace and
- configure your machine so you can use the Azure Machine Learning Services from your local notebook.

Besides creating CPU and GPU clusters, it also shows creating an Azure Kubernetes Service with Azure Machine Learning Services.

To improve readability, the notebook does not use many distracting explanations. At least it tries to.

If you are looking for sample notebooks with more explanations or samples that show features not shown here, see the official samples repo [here](https://github.com/Azure/MachineLearningNotebooks). That repo is updated more often than this one.

The official documentation of the Azure Machine Learning Service is [here](https://docs.microsoft.com/en-us/azure/machine-learning/service). Note that it also contains very helpful how-to guides.

Azure Machine Learning Services is mostly Python but can run R code as well, eg. using the [rpy2](https://rpy2.readthedocs.io/en/version_2.8.x) package (until official R support is out). See my example for a scoring web service in R [here](https://github.com/timoklimmer/run-r-on-amls). Running R jobs on an Azure Machine Learning Compute Cluster or even in a pipeline works similar. Unfortunately, I didn't write a sample for it yet. Before you begin with R on AMLS you should also check if official R support is there meanwhile.

Note that you can do a lot with Azure Machine Learning Services already on your local notebook (saving costs). However in (almost) any case, for using Azure Machine Learning Services you will need an Azure subscription.

## Installation

### Azure Subscription

Using the Azure Machine Learning Service requires that you have access to an Azure subscription (or at least a resource group within that subscription) and know the ID of this subscription. Subscription IDs look like `d11f4dfd-a10a-4041-beca-9f47839dc7ed`.

Besides, your account needs sufficient permissions. If you don't know what an account is: it looks like an e-mail address and likely contains the name of your company. For the steps described here, a Contributor role on the subscription or a resource group should suffice.

In addition, it might happen that some services ("providers") need to be registered on your subscription before you use these the first time. For instance, a *subscription owner* - not necessarily you - has to register the Azure Container Instances (ACI) service. See below for more details.

In case you have no access to an Azure subscription you can try a [trial subscription](https://azure.microsoft.com/en-us/free). But be aware that these come with some limitations such as being time-bombed, a limited budget and no access to all virtual machine sizes.


### Conda

1. Get a Windows, macOS or Linux machine.
2. Ensure that your NVIDIA graphics driver is up-to-date if you have a GPU that you want to use.
3. Ensure that you have either [Anaconda](https://www.anaconda.com/distribution) or [Miniconda](https://docs.conda.io/en/latest/miniconda.html) installed. Miniconda is preferred - it's more lightweight.
4. Open the Anaconda Prompt. Steps are different, depending on what you selected during installation. If your command prompt begins with `(base)`, eg. "(base) C:\Users\administrator", you are right.
5. Create a new conda environment by running `conda create -n <env_name, eg. azureml> python=3.6`.
6. Activate the new environment by running `conda activate <the env_name you chose before>`.
7. Ensure conda is up-to-date by running `conda update -n base -c defaults conda`.
8. Set the conda-forge channel as the default channel by running `conda config --add channels conda-forge`.
9. Run `conda install pip` to avoid integration issues with pip and getting an inconsistent environment.

Note: Although you could, I do not recommend to use the `base` environment. Separate environments improve manageability and can avoid many headaches. **From now on, you always have to activate the environment you created before in order to use Azure Machine Learning Services.**


### Packages

#### *Proxy*
If your computer needs to communicate with the internet through a proxy ensure that your command line shell uses the proxy. Often, this is done through setting a special environment variable (HTTP_PROXY and HTTPS_PROXY). For more infos contact your IT administrator/helpdesk. Conda has also a mechanism to set environment variables whenever an environment is activated. See [here](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#saving-environment-variables) for more details.

#### *conda environment*
**Ensure that you run all commands below with your new conda environment activated!**

#### Git (strongly recommended)
Don't even dare to not install Git. If you haven't installed it yet (check by running `git version`), install it. Get the right installer or command to install it [here](https://git-scm.com/downloads).

#### node.js (optional)
Run `conda install nodejs` so you can install extensions for Jupyter Lab.

#### Jupyter Notebook/Jupyter Lab (optional)
Run
```
conda install jupyterlab
conda install jupyter_nbextensions_configurator
```
This will install the new Jupyter Lab as well as the classic Jupyter Notebooks.

Notes:

- AzureML will install some widgets (custom UI controls) for Jupyter and Jupyter Labs. At the time of writing, due to some changes in Jupyter Lab, these widgets do not work in Jupyter Labs currently (until fixed). If you want to use them today you either have to use a classic Jupyter notebook, Jupyter Lab version 0.3x or the Jupyter Lab version provided by an Azure ML Notebook VM.

- The *jupyter_nbextensions_configurator* package adds some nice add-ins to classic Jupyter. Open the *nbextensions* tab in Jupyter to activate them - eg. to get auto-completion or a table of contents on the side.

#### azureml-sdk
Run `pip install azureml-sdk[accel-models,automl,contrib,explain,notebooks,services,tensorboard]`

See [here]([https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py) for more details, esp. on the different package extras.

#### Azure CLI (optional)
Run `pip install azure-cli` to install the Azure CLI for managing Azure from the command line.

#### Azure ML CLI extension (optional)
Not necessarily required but if you want to run some Azure Machine Learning Service-related tasks from the command line - eg. if you want to do things out of PyCharm - there is a CLI extension. See [here](https://docs.microsoft.com/azure/machine-learning/service/reference-azure-machine-learning-cli) for more details.

To install run `az extension add -n azure-cli-ml`.

`az ml -h` then shows a help for all sub commands.

To update the extension run `az extension update -n azure-cli-ml`.

The Azure ML CLI has a lot of features, eg. can be used to create clusters, manage datasets and datastores, ... However, in some cases the Azure ML SDK for Python is ahead, so you might need to use the Python SDK to achieve certain things.


#### Further optional packages
Install whatever packages you need and like, eg.
- Run `conda install tqdm matplotlib` to install the popular tqdm and matplotlib packages (often required by sample code).
- Run `conda install pylint black yapf` to improve code editing.
- Run `conda install rope` so VS.Code does not ask for an install if a rename symbol refactoring operation is done.
- Run `jupyter labextension install @jupyter-widgets/jupyterlab-manager` to get Jupyter/IPython widget support in Jupyter Labs
- Run `jupyter labextension install microsoft-mli-widget` to install the Azure Machine Learning widgets for Jupyter Labs
- Run `jupyter labextension install @jupyterlab/git`, `pip install --upgrade jupyterlab-git` and `jupyter serverextension enable --py jupyterlab_git` to enable Git support in Jupyter Labs.
- Run `conda install tensorflow-gpu` to install the GPU version of TensorFlow (remove -gpu if you don't have a GPU)
- Run `conda install keras-gpu` to install the GPU version of Keras (remove -gpu if you don't have a GPU)
- Run `conda install pytorch torchvision -c pytorch` to install PyTorch
- Run `pip install gym` to install the minimum version of OpenAI Gym (see the [documentation](https://github.com/openai/gym#installation) for more infos)

### Optional: Azure Storage Explorer
If you are using [Azure Storage](https://azure.microsoft.com/services/storage) (which includes Blob Containers, File Shares, Queues, Tables), you might want to install [Azure Storage Explorer](https://azure.microsoft.com/features/storage-explorer) as well. Azure Storage Explorer lets you see and edit the content of your Azure Storage. You can download the installer from [here](https://azure.microsoft.com/features/storage-explorer). 

### Optional: Docker
Not necessarily needed but can help in some cases, eg. to debug things. Get and install Docker from [here](https://docs.docker.com/install/). Docker should be configured to run Linux containers. If on Windows: you are right if the context menu behind the Docker icon on right of your taskbar has the option "*Switch to Windows containers...*".

### Login to Azure
- Ensure you are in your new environment and
- Run `az login` to log in to Azure.

If you have access to multiple subscriptions, you might want to look if you are currently working against the correct subscription.

- Run `az account show` to see which subscription is active and
- if needed: Use `az account set --subscription <name or id>` to change the active subscription.


### Option 1: Juypter
Now that Jupyter and all required packages for using Azure Machine Learning Services is installed you can open this notebook in Jupyter to proceed.

To do that:
- download the file to an arbitrary directory onto your computer
- if needed: activate your new environment
- in your Anaconda Prompt: go to the directory with the notebook file
- run `jupyter notebook`
- open the file of this notebook.

Pros/Cons w.r.t. the coding experience:

(+) It's good for documentation-intensive code.<br>
(+) Most people use or at least know Jupyter.<br>
(+) Many examples are published in Jupyter notebooks.<br>
(+) Code output can be stored. That makes sharing results easier.<br>
(+) It's easy to switch between languages.<br>
(+) Sometimes there are special widgets (UI controls) that run in Jupyter only.<br>
(-) Editing files is cumbersome.<br>
(-) The same with source code control.<br>
(-) Running individual code blocks is easier in other IDEs such as VS.Code.


### Option 2: Jupyter Lab
As an alternative to Jupyter you can also use (the free) [Jupyter Lab](https://jupyterlab.readthedocs.io/en/stable) which is the next-generation web-based user interface for Project Jupyter. If you have installed it before (see section: *Optional packages*), you can start it by running `jupyter lab`.

The pros and cons regarding the coding experience are similar to Jupyter. However, one great advantage is that it has a file explorer.


### Option 3: (Azure) Databricks
[Azure Databricks](https://azure.microsoft.com/de-de/services/databricks/) is a managed Spark service and has its own notebook-based user interface. The provising of an Azure Databricks environment that uses Azure Machine Learning services is out of scope for this notebook and is different than the steps described here. See [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment#azure-databricks) for Azure Databricks-specific instructions.

The pros and cons regarding the coding experience are similar to Jupyter.


### Option 4: VS.Code
Another option to interact with the Azure Machine Learning Service is to use the free [Visual Studio Code](https://code.visualstudio.com).

Before you use VS.Code with the Azure Machine Learning Services, ensure you have added at least the following extensions to VS.Code:
- the Python extension as well as
- the Azure Machine Learning extension

Pros/Cons w.r.t. the coding experience:

(+) A general IDE not only for data science / machine learning tasks.<br>
(+) Supports many programming languages.<br>
(+) Can import/export Jupyter notebooks.<br>
(+) Very good debugging experience.<br>
(+) Very good integration with Git and other source code control systems<br>
(+) Very good autocompletion, syntax highlighting etc.<br>
(-) Reading and writing code documentation is not as comfortable as in Jupyter.<br>
(-) Can run selected code in integrated Jupyter but some widgets incl. the AzureML ones are not supported yet


### Option 5: Some other IDE
Last but not least you can also use any other Python IDE such as PyCharm. Only requirement is that you run your Python code in the environment you created previously. If your IDE does not have an integration with Azure Machine Learning Services, you can always submit your scripts etc. through running Azure CLI commands directly.

### Azure Machine Learning Workspace

#### Configuration

In [ ]:
import os

### Azure general

# Subscription id
# note: As a best practice, I intentionally fetch my Azure subscription ID from an environment variable to avoid that it
#       will be stored hard-coded in this notebook. Feel free to adopt or put in your subscription id in here directly.
#       Another alternative is to put all the settings in a config file (see official examples repo for more details).
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")

# Resource Group
resource_group = "azureml-spike-rg"
resource_group_create_if_not_exists = True

# Workspace
workspace_name = "OurWorkspace"

# Region
# note: check https://azure.microsoft.com/en-us/global-infrastructure/services/?products=machine-learning-service for
#       currently available and planned regions
workspace_region = "westeurope"

# notes: - see https://docs.microsoft.com/azure/virtual-machines/linux/sizes for all available machine sizes.
#        - some VM sizes have to be enabled through the Azure Portal / Microsoft Support before they can be used.

### Batch compute

## AML Compute clusters

# CPU cluster
cpu_cluster_name = "cpu-cluster"
cpu_cluster_vm_size = "STANDARD_D3_V2"
cpu_cluster_min_nodes = 0
cpu_cluster_max_nodes = 4

# GPU cluster
gpu_cluster_name = "gpu-cluster"
gpu_cluster_vm_size = "STANDARD_NC6"
gpu_cluster_min_nodes = 0
gpu_cluster_max_nodes = 4

### Scoring

## AKS cluster
# notes: - don't forget to shutdown the cluster when you are done with your explorations.
#          a cluster with multiple nodes that is not used in production can become expensive.
#        - the cluster name has to be unique across Azure
#        - see https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.aks.akscompute
#          (method provisioning_configuration()) for more detailed infos on the parameters
# name and size
aks_cluster_name = "aks-cluster"
aks_cluster_agent_count = 3
aks_cluster_vm_size = "STANDARD_D3_V2"
# SSL/TLS
aks_cluster_use_certificate_from_microsoft = True
# certificate from Microsoft
aks_cluster_leaf_domain_label = aks_cluster_name
# custom certificate
# notes: - no values needed if a certificate from Microsoft is used
#        - the cname in the certificate must match the DNS name of the cluster
#        - be cautious with self-signed certificates, some applications need a real certificate to work properly
aks_cluster_ssl_cname = None
aks_cluster_ssl_cert_pem_file = None
aks_cluster_ssl_key_pem_file = None
# Azure region
aks_cluster_location = None
# VNET
aks_cluster_vnet_resourcegroup_name = None
aks_cluster_vnet_name = None
aks_cluster_subnet_name = None
aks_cluster_service_cidr = None
aks_cluster_dns_service_ip = None
aks_cluster_docker_bridge_cidr = None
aks_cluster_cluster_purpose = None

## Azure Container Instances
# notes: - ACI does not need any upfront setup, hence there are no settings to configure EXCEPT ensuring that
#          ACI is registered with your subscription. See the code below for details.
#        - ACI fits well for dev/test scenarios. It is not recommended for production use.

## Azure App Service / Web Apps for Containers (preview)
# notes: - see https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-app-service for details
#        - this option is currently in preview and therefore has some limitations, eg. it's not clear if it will be
#          released at all
#        - however, since it's App Service, a lot of its features can be used incl.
#            * Azure AD auth (incl. two-factor auth)
#            * Autoscaling
#        - use the App Service as deployment target if you create web services that have spikes in their usage, eg.
#          at the end of a month. besides, App Service can be a quick means to provide a web service that requires
#          multi-factor auth against Azure AD or another identity provider.
#        - there is no feature in Azure Machine Learning Services yet which deploys to the App Service,
#          hence you have to follow the steps described in the link above to do the deployment.
#          if you need a programmatical deployment before it's available in Web Apps you can use the
#          Azure CLI to do get the deployment automation.

#### Create Workspace (if not exists)

In [ ]:
# see https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace for more details
from azureml.core import Workspace
from azureml.exceptions import ProjectSystemException

try:
    aml_workspace = Workspace(subscription_id=subscription_id,
                              resource_group=resource_group,
                              workspace_name=workspace_name)
    print("Workspace '{}' exists already.".format(workspace_name))
except ProjectSystemException:
    print("Workspace '{}' does not exist yet.".format(workspace_name))
    print("Creating workspace...".format(workspace_name))
    aml_workspace = Workspace.create(
        name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,
        location=workspace_region,
        create_resource_group=resource_group_create_if_not_exists)

print("Done.")

Note: For some reason the used try..catch above does not work as expected if the workspace did not exist before. **The execution of the script was still successful if the output ends with "Done."**

As an alternative, the workspace can also be created using Azure ML's CLI (see above for installation, [here](https://docs.microsoft.com/de-de/cli/azure/ext/azure-cli-ml) for the documentation).

Example: `az ml workspace create -w OurWorkspace -g azureml-spike-rg -l westeurope -y`

Be sure to adjust the parameters above as required and run `az ml workspace create -h` for help on the syntax.

#### Optional: Register datastore(s)

In Azure Machine Learning Services, a *datastore* can be understood as a reference to a datasource. This reference can be used at some places in Azure Machine Learning Services to make data more easily available to scripts. Multiple datasource types are supported incl. Blob Store, File Share, Data Lake, Data Lake Gen2, Azure SQL, PostgreSQL, ...

An example code to register a dataset would be:

```python
import azureml.core
from azureml.core import Workspace, Datastore

datastore = Datastore.register_azure_blob_container(workspace=aml_workspace, 
                                                    datastore_name='your datastore name', 
                                                    container_name='your azure blob container name',
                                                    account_name='your storage account name', 
                                                    account_key='your storage account key',
                                                    create_if_not_exists=True)
```

See [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data) for more details.

As an alternative, datastores can also be registered through the Azure ML CLI. Run `az ml datastore -h` to get details.

Note that datastores can ease things but they are not required. You can also use regular Python code to get data from different sources such as Blob Store.

#### Optional: Register dataset(s)

*Datasets* are another way to provide data to scripts more easily.

An example for registering a dataset would be

```python
from azureml.core.workspace import Workspace
from azureml.core.datastore import Datastore
from azureml.core.dataset import Dataset

datastore_name = 'your datastore name'

# retrieve an existing datastore in the workspace by name
datastore = Datastore.get(aml_workspace, datastore_name)

# create a TabularDataset from multiple paths in datastore
datastore_paths = [
                  (datastore, 'weather/2018/11.csv'),
                  (datastore, 'weather/2018/12.csv'),
                  (datastore, 'weather/2019/*.csv')
                 ]
weather_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

# create a TabularDataset from a delimited file behind a public web url
web_path ='https://dprepdata.blob.core.windows.net/demo/Titanic.csv'
titanic_ds = Dataset.Tabular.from_delimited_files(path=web_path)

# preview the first 3 rows of titanic_ds
titanic_ds.take(3).to_pandas_dataframe()
```

Note that there a lot more options with datasets, eg. profiling or sampling the data before training. Through the dataset feature you can even hold different version of your data!

For more details see [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-create-register-datasets) and [here](https://github.com/Azure/MachineLearningNotebooks/tree/master/work-with-data/datasets).

#### Optional: CPU Cluster (if not exists)

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    cpu_cluster = ComputeTarget(workspace=aml_workspace, name=cpu_cluster_name)
    print("Cluster '{}' exists already.".format(cpu_cluster_name))
except ComputeTargetException:
    print("Cluster '{}' does not exist yet.".format(cpu_cluster_name))
    print("Creating cluster...".format(cpu_cluster_name))
    cpu_cluster = ComputeTarget.create(
        aml_workspace, cpu_cluster_name,
        AmlCompute.provisioning_configuration(vm_size=cpu_cluster_vm_size,
                                              min_nodes=cpu_cluster_min_nodes,
                                              max_nodes=cpu_cluster_max_nodes))
    cpu_cluster.wait_for_completion(show_output=True)

print("Done.")

As an alternative, the AML Compute cluster can be created using Azure ML's CLI (see above for installation, [here](https://docs.microsoft.com/de-de/cli/azure/ext/azure-cli-ml) for the documentation).

Example: `az ml computetarget create amlcompute -n cpu-cluster --vm-size STANDARD_D2_V2 --min-nodes 0 --max-nodes 4 -g azureml-spike-rg -w OurWorkspace`

Be sure to adjust the parameters above as required and run `az ml computetarget create amlcompute -h` for help on the syntax.

Mind the VM size - in some scenarios it makes sense to have multiple CPU clusters. Eg. a cluster with medium-perf VM sizes and a cluster with high-perf, many cores machine.

#### Optional: GPU Cluster (if not exists)

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    gpu_cluster = ComputeTarget(workspace=aml_workspace, name=gpu_cluster_name)
    print("Cluster '{}' exists already.".format(gpu_cluster_name))
except ComputeTargetException:
    print("Cluster '{}' does not exist yet.".format(gpu_cluster_name))
    print("Creating cluster...".format(gpu_cluster_name))
    gpu_cluster = ComputeTarget.create(
        aml_workspace, cpu_cluster_name,
        AmlCompute.provisioning_configuration(vm_size=gpu_cluster_vm_size,
                                              min_nodes=gpu_cluster_min_nodes,
                                              max_nodes=gpu_cluster_max_nodes))
    gpu_cluster.wait_for_completion(show_output=True)

print("Done.")

As an alternative, the AML Compute cluster can be created using Azure ML's CLI (see above for installation, [here](https://docs.microsoft.com/de-de/cli/azure/ext/azure-cli-ml) for the documentation).

Example: `az ml computetarget create amlcompute -n cpu-cluster --vm-size STANDARD_NC6 --min-nodes 0 --max-nodes 4 -g azureml-spike-rg -w OurWorkspace`

Be sure to adjust the parameters above as required and run `az ml computetarget create amlcompute -h` for help on the syntax.

Mind the VM size. The NC6 machines are the slowest GPU machines available in Azure. Azure has a lot more powerful GPU machines.

#### Optional: Azure Kubernetes Service (if not exists)

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException

try:
    aks_cluster = ComputeTarget(workspace=aml_workspace, name=aks_cluster_name)
    print("Cluster '{}' exists already.".format(aks_cluster_name))
except ComputeTargetException:
    print("Cluster '{}' does not exist yet.".format(aks_cluster_name))
    print("Creating cluster...".format(aks_cluster_name))
    provisioning_configuration = AksCompute.provisioning_configuration(
        agent_count=aks_cluster_agent_count,
        vm_size=aks_cluster_vm_size,
        ssl_cname=aks_cluster_ssl_cname,
        ssl_cert_pem_file=aks_cluster_ssl_cert_pem_file,
        ssl_key_pem_file=aks_cluster_ssl_key_pem_file,
        location=aks_cluster_location,
        vnet_resourcegroup_name=aks_cluster_vnet_resourcegroup_name,
        vnet_name=aks_cluster_vnet_name,
        subnet_name=aks_cluster_subnet_name,
        service_cidr=aks_cluster_service_cidr,
        dns_service_ip=aks_cluster_dns_service_ip,
        docker_bridge_cidr=aks_cluster_docker_bridge_cidr,
        cluster_purpose=aks_cluster_cluster_purpose)
    
    if aks_cluster_use_certificate_from_microsoft:
        provisioning_configuration.enable_ssl(
            leaf_domain_label=aks_cluster_leaf_domain_label)
    else:
        provisioning_configuration.enable_ssl(
            ssl_cert_pem_file=aks_cluster_ssl_cert_pem_file,
            ssl_key_pem_file=aks_cluster_ssl_key_pem_file,
            ssl_cname=aks_cluster_ssl_cname)
        
    aks_cluster = ComputeTarget.create(
        workspace=aml_workspace,
        name=aks_cluster_name,
        provisioning_configuration=provisioning_configuration)

    aks_cluster.wait_for_completion(show_output=True)
    print(aks_cluster.provisioning_state)
    print(aks_cluster.provisioning_errors)

print("Done.")

As an alternative, the AKS cluster can be created using Azure ML's CLI (see above for installation, [here](https://docs.microsoft.com/de-de/cli/azure/ext/azure-cli-ml) for the documentation).

Example: `az ml computetarget create aks -n aks-cluster --agent-count 3 -s Standard_D3_v2 -g azureml-spike-rg -w OurWorkspace --cluster-purpose "DevTest"`

Be sure to adjust the parameters above as required and run `az ml computetarget create aks -h` for help on the syntax.

**Important:** In case you bring your own SSL certificates, you need to update the DNS name label of your cluster after the AKS cluster is deployed. See [here](https://docs.microsoft.com/de-de/azure/machine-learning/service/how-to-secure-web-service) for more details.

**Important 2:** Mind that SSL certificates can and will expire. See [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-secure-web-service?view=azure-ml-py) for instructions on how to update certificates.

#### Optional: Azure Container Instances

As described in the configuration section above, there is no need to setup a cluster or service upfront. However, ACI needs to be registered with your subscription. This can be done by having a SUBSCRIPTION OWNER - not necessarily you - run the following command.

*az* commands can be run through multiple options. One is the [Azure CLI](https://docs.microsoft.com/cli/azure) (see above), another one is the [Azure Cloud Shell](https://azure.microsoft.com/de-de/features/cloud-shell). The cloud shell is a good option if you don't have an up-to-date version of the Azure CLI installed on your system and quickly need to run a shell command. You can open the Cloud Shell from the [Azure Portal](portal.azure.com). Click on the shell icon on top right to open it. 

You can omit the *subscription* parameter if the subscription is already the active one. (Check with `az account show` if necessary.) To register or check a provider with another name just replace "Microsoft.ContainerInstance" against the name of the other provider.

In [ ]:
!az provider register -n Microsoft.ContainerInstance --subscription <id of your subscription>

To check if a provider is already registered, you can use the following command (replace the provider name as necessary)

In [ ]:
!az provider show -n Microsoft.ContainerInstance -o table --subscription <id of your subscription>

#### Optional: Azure App Service

Follow the steps described [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-app-service) to deploy a containerized model to the Azure App Service.

### Optional: Azure DevOps

Azure DevOps is currently out-of-scope for this notebook. But if you have no Azure DevOps account yet, check [this](https://dev.azure.com).

Setting up Azure DevOps for ML workloads is not that easy but luckily, there is a good repo which shows how to do it [here](https://github.com/microsoft/MLOpsPython).

### Optional: Azure API Management

Depending on your scenario, esp. for production scenarios, you might want to put an [Azure API Management](https://azure.microsoft.com/de-de/services/api-management) service in front of your scoring web service. The installation/configuration of it is out-of-scope of this notebook.

## Cleanup

As we have everything in a resource group we can simply delete the resource group for the cleanup.

- If you have the Azure CLI installed, you can run the `az group delete -n <name of resource_group>` command.
- Another option would be the [Azure Portal](portal.azure.com) or
- [Azure's Python SDK](https://docs.microsoft.com/en-us/azure/python/python-sdk-azure-install)